### Gloden cross strategy

In [1]:
import time
from datetime import datetime

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### import csv

In [2]:
data = pd.read_csv('EURUSD_15m_2022_01_03.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
3501,2021-12-31 20:45:00+00:00,1.138952,1.139212,1.138174,1.138174,1.138174,0
3502,2021-12-31 21:00:00+00:00,1.138045,1.138304,1.137915,1.138174,1.138174,0
3503,2021-12-31 21:30:00+00:00,1.138174,1.138174,1.138045,1.138045,1.138045,0
3504,2021-12-31 21:45:00+00:00,1.138174,1.138563,1.137010,1.137268,1.137268,0
3505,2021-12-31 22:00:00+00:00,1.137268,1.137268,1.137268,1.137268,1.137268,0


In [3]:
df = data.copy()

### Create strategy

In [4]:
gloden_cross = ta.Strategy(
    name = 'Gloden cross',
    description = 'Gloden cross by EMA 2 Line',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'ema', 'length': 10},
        {'kind': 'ema', 'length': 50}
    ]
)
df.ta.strategy(gloden_cross)

### Create signal

In [47]:
class BacktestGlodenCross:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        
        self.current_action = 'close'
        self.atr_len = params['atr_len']
        self.ema_1 = params['ema_1']
        self.ema_2 = params['ema_2']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def backtest(self):
        n_bars = 3
        gloden_cross = ta.Strategy(
            name = 'Gloden cross',
            description = 'Gloden cross by EMA 2 Line',
            ta = [
                {'kind': 'ema', 'length': 200},
                {'kind': 'ema', 'length': self.ema_1},
                {'kind': 'ema', 'length': self.ema_2},
                {'kind': 'atr', 'length': self.atr_len}
            ]
        )
        self.df.ta.strategy(gloden_cross)
        self.df.dropna()
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
            
                if (row['EMA_'+str(self.ema_1)] > row['EMA_'+str(self.ema_2)]) and (row['low'] > row['EMA_'+str(200)]):
                    # check if previous n bars ema_1 < ema_2 then buy.
                    if index > n_bars:
                        for pre_index, pre_row in self.df.iloc[index-n_bars:index].iterrows():
                            if (pre_row['EMA_'+str(self.ema_1)] < pre_row['EMA_'+str(self.ema_2)]):
                                row['action'] = 'buy'
                                self.order['action'] = 'buy'
                                self.order['open time'] = row['timestamp']
                                self.order['open'] = row['open']
                                self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr_len)]
                                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                                self.current_action = 'buy'
                        
                elif (row['EMA_'+str(self.ema_1)] < row['EMA_'+str(self.ema_2)]) and (row['high'] < row['EMA_'+str(200)]):
                    # check if previous n bars ema_1 > ema_2 then sell.
                    if index > n_bars:
                        for pre_index, pre_row in self.df.iloc[index-n_bars:index].iterrows():
                            if (pre_row['EMA_'+str(self.ema_1)] > pre_row['EMA_'+str(self.ema_2)]):
                                row['action'] = 'sell'
                                self.order['action'] = 'sell'
                                self.order['open time'] = row['timestamp']
                                self.order['open'] = row['open']
                                self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr_len)]
                                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                                self.current_action = 'sell'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema_1: {self.ema_1}, ema_2: {self.ema_2}, atr: {self.atr_len}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [48]:
eurusd = BacktestGlodenCross({
    'atr_len': 14,
    'ema_1': 10,
    'ema_2': 50,
    'pip_value': 0.001,
    'rr': 1.5,
    'df': df
})
eurusd.backtest()

{'win_rate': 45.33,
 'info': 'total orders: 75\nema_1: 10, ema_2: 50, atr: 14, gain: 7.5\n',
 'total_order': 75,
 'rr': 1.5,
 'gain': 7.5,
 'unknow': []}

In [49]:
"""
Grid parameter example
line 1: 10 -> 40
line 2: 30 -> 80
condition line 1 < line 2 < line 3
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            if( i < j ):
                params.append([i, j])
    return params
params_list = create_grid_params(10, 40, 30, 80)
len(params_list)

1515

In [50]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 2000
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BacktestGlodenCross({'atr_len': 14, 'ema_1': param[0], 'ema_2': param[1], 'pip_value': 0.001, 'rr': 1.5, 'df': df})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 30:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 5 to 5
win rate: 50.0 %
total orders: 98
ema_1: 10, ema_2: 30, atr: 14, gain: 24.5

